In [ ]:
# SELF SUPERVISED LEARNING
%cd ~/Documents/TESI/TESI/
%load_ext autoreload
%autoreload 2
from utils.setup_functions import set_environment_flags
# Example usage:
environment_flags = set_environment_flags()
kaggle,gdrive,linux = environment_flags["kaggle"], environment_flags["gdrive"], environment_flags["linux"]
from utils.reproducibility_functions import set_global_seed
set_global_seed(42)
import os
import re
import tifffile
import glob
import random
import torch
import pytorch_lightning as pl
print(torch.__version__)
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from configs.ConfigLoader import ConfigLoader
from utils.train_functions import (
# train_epoch,
# val_epoch,
# print_model_summary,
# plot_cv_results,
# train_epoch_mixUp,
# print_layers,
# oversample_minority,
# undersample_majority,
# freeze_layers_up_to,
# freeze_layers_up_to_progressive_ft,
train_epoch_vit,
val_epoch_vit,
    )

from utils.test_functions import evaluate_model
from utils.data_visualization_functions import plot_confusion_matrix, show_misclassified_images, plot_roc_curve

import utils.transformations_functions as tf
# Removed redundant import: from configs.ConfigLoader import ConfigLoader

from classes.ModelManager import ModelManager
import monai
print(monai.__version__)
#import tifffile
#from monai.networks.nets import DenseNet121
# import torch.nn.functional as F
# from monai.visualize.class_activation_maps import GradCAMpp,GradCAM  
#kaggle = input("Are you on Kaggle? Enter 'T' for True or 'F' for False: ")
kaggle = False
if gdrive:
    import os

    # Change to the desired directory:
    os.chdir('/content/drive/MyDrive/TESI/TESI/notebooks')

    # Verify the current directory:
    print(os.getcwd())

# start mlflow ui
from utils.mlflow_functions import *
from utils.directory_functions import *

tracking_uri = get_tracking_uri(gdrive,kaggle,linux)
mlflow.set_tracking_uri(tracking_uri)
start_mlflow_ui(tracking_uri) # start mlflow ui
from utils.directory_functions import get_data_directory, get_base_directory
num_input_channels = int(input("Enter the number of input channels (3 or 4): "))

from utils.directory_functions import get_data_and_base_directory
data_dir, base_dir = get_data_and_base_directory(environment_flags["kaggle"], environment_flags["gdrive"], environment_flags["linux"], num_input_channels=num_input_channels)
import ipywidgets as widgets
from IPython.display import display

CLASS_NAME_SETS = {
    "MSA vs Control": ["MSA", "control"],
    "MSA vs PD": ["MSA", "PD"],
    "MSA-P vs MSA-C": ["MSA-P", "MSA-C"],
    "MSA-P vs PD": ["MSA-P", "PD"],
    "PD vs MSA-P vs MSA-C": ["PD", "MSA-P", "MSA-C"]
}

dropdown = widgets.Dropdown(
    options=list(CLASS_NAME_SETS.keys()),
    value="MSA vs PD",
    description='Class Set:',
    style={'description_width': 'initial'}
)

def on_dropdown_change(change):
    """
    Update the class_names variable when the dropdown selection changes.
    """
    global class_names
    if change['type'] == 'change' and change['name'] == 'value':
        class_names = CLASS_NAME_SETS[change['new']]
        print(f"class_names set to: {class_names}")


class_names = CLASS_NAME_SETS[dropdown.value]

dropdown.observe(on_dropdown_change)

display(dropdown)
## Paths of ALL images into a numpy array without labels used for SSL
all_images_folder_path = os.path.join(data_dir, "ALL")
all_images_paths = glob.glob(os.path.join(all_images_folder_path, "*.tif"))
all_images_paths = np.array(all_images_paths)
print("Number of images in ALL folder:", len(all_images_paths))
# Create a dictionary mapping each class to its directory
class_dirs = {}
three_classes = (len(class_names) == 3)

for class_name in class_names:
    class_dirs[class_name] = os.path.join(data_dir, class_name)
    
print(class_dirs)
if three_classes:
    class2_name, class1_name, class0_name = class_names
    class2_dir, class1_dir, class0_dir = class_dirs.values()
else:
    class1_name, class0_name = class_names
    class1_dir, class0_dir = class_dirs.values()

print("Class directories:")
print(class_dirs)

# Dictionaries to store image paths and counts for each class
images_paths_dict = {}
counts_dict = {}

# Loop over classes to process each folder
for class_name in class_names:
    class_dir = class_dirs[class_name]
    image_paths = sorted(glob.glob(os.path.join(class_dir, "*.tif")))
    
    # Check if images were found; otherwise raise an error
    if not image_paths:
        raise FileNotFoundError(f"No TIFF image file found in {class_dir}")
    
    # Count occurrences of 'gh' and 'vaso' in the filenames (using .lower() for case insensitivity)
    gh_count = sum('gh' in os.path.basename(path).lower() for path in image_paths)
    vaso_count = sum('vaso' in os.path.basename(path).lower() for path in image_paths)
    print(f"{class_name} images (before filtering): 'gh' count: {gh_count}, 'vaso' count: {vaso_count}")
    
    # Filter out images that contain 'vaso' (if needed)
    image_paths = [path for path in image_paths if 'vaso' not in os.path.basename(path).lower()]
    gh_count_after = sum('gh' in os.path.basename(path).lower() for path in image_paths)
    vaso_count_after = sum('vaso' in os.path.basename(path).lower() for path in image_paths)
    print(f"After removing 'vaso', {class_name} images: 'gh' count: {gh_count_after}, 'vaso' count: {vaso_count_after}")
    
    # Store the filtered image paths and counts for later use
    images_paths_dict[class_name] = image_paths
    counts_dict[class_name] = {"gh_count": gh_count_after, "vaso_count": vaso_count_after}

# Visualize the number of 'gh' counts per class in a bar chart
plt.figure()
plt.xlabel("Class")
plt.ylabel("Number of 'gh' occurrences")
plt.title("Number of 'gh' occurrences per class")
bar_heights = [counts_dict[cn]["gh_count"] for cn in class_names]
bar_colors = ['red', 'blue', 'lightblue']
plt.bar(class_names, bar_heights, color=bar_colors)
plt.show()

# --- Debug: Check image shapes after initial loading ---
print("\nChecking image shapes:")
for class_name, image_paths in images_paths_dict.items():
    for path in image_paths:
        img = tifffile.imread(path)  # Read image as a numpy array
        print(f"{class_name} image: {os.path.basename(path)}  dtype: {img.dtype}, shape: {img.shape}")

# Combine image paths and labels for the three classes; 
# the label here is simply the index of the class in class_names (0, 1, 2)
combined = [] # List to store tuples of (image_path, label)
for label, class_name in enumerate(class_names):
    for path in images_paths_dict[class_name]:
        combined.append((path, label))
# print("\nSample of combined image paths and labels:", combined[:5])
# random.shuffle(combined)  # Shuffle the combined list to mix classes

# Optionally, determine the minority label for resampling purposes
counts = {label: len(images_paths_dict[class_name]) for label, class_name in enumerate(class_names)}
minority_label = min(counts.keys(), key=lambda k: counts[k])
print(f"\nMinority label for resampling purposes: {minority_label}")

# Unzip the combined list back into separate tuples (if needed)
images_paths, labels = zip(*combined)
print("\nSample of image paths:", images_paths[:5])
print("Total images found:", len(combined))

# Optionally, convert to NumPy arrays (helpful for further processing or k-fold splitting)
images_paths_np = np.array(images_paths)
labels_np = np.array(labels)
print("\nSample of image paths (NumPy):", images_paths_np[:5])
print((labels_np))
#print(X)
# --- Your Split Logic for 50/50 distribution in test set ---

print("Original dataset size:", len(images_paths_np))

# Find unique labels and their counts in the original dataset
unique_labels, counts = np.unique(labels_np, return_counts=True)
original_distribution = dict(zip(unique_labels, counts))
print(f"Original label distribution: {original_distribution}")

# Determine the maximum possible size for a balanced test set per class
# This is limited by the count of the smallest class
if len(unique_labels) > 1:
    min_class_count = min(counts)
    # We want a balanced test set, so take 'min_class_count' samples from each class
    test_samples_per_class = min_class_count
    total_balanced_test_size = test_samples_per_class * len(unique_labels)

    print(f"\nAiming for a balanced test set with {test_samples_per_class} samples per class.")
    print(f"Total balanced test set size will be: {total_balanced_test_size}")

    test_indices = []
    train_indices = []

    # Iterate through each class to split
    for label in unique_labels:
        # Get the indices in the original array that correspond to the current class
        # print ( labels_np == label) # returns a boolean array
        class_indices = np.where(labels_np == label)[0]  #use the boolean array to get the indices where cond is true 
        # print(f"\nClass {label} indices: {class_indices}") #retuns the indices of the class in the original array and the boolarray so we use [0] to get the indices

        # Randomly select a fixed number of indices for the test set from this class
        # Use np.random.choice with replace=False for sampling without replacement
        # Set a random_state for reproducibility if needed
        rng = np.random.default_rng(42) # Use new random generator recommended over np.random.seed
        test_class_indices = rng.choice(
            class_indices,
            size=test_samples_per_class,
            replace=False
        )
        test_indices.extend(test_class_indices)

    # Convert lists of indices to NumPy arrays
    test_indices = np.array(test_indices)
    train_indices = np.array(train_indices)

    # Shuffle the indices to mix up the classes in the final arrays (optional but good practice)
    # rng.shuffle(test_indices)
    # rng.shuffle(train_indices)

    balanced_test_images_paths = images_paths_np[test_indices]
    balanced_test_true_labels = labels_np[test_indices]
    print(f"Test set size: {len(balanced_test_images_paths)}")

    # Verify the test set distribution
    test_unique_labels, test_counts = np.unique(balanced_test_true_labels, return_counts=True)
    test_distribution = dict(zip(test_unique_labels, test_counts))
    print(f"\nTest set distribution: {test_distribution}")


    plt.figure(figsize=(6, 4))
    # Use test_unique_labels and test_counts for the bar plot
    labels_for_plot = [class_names[label] if 'class_names' in locals() else f"Label {label}" for label in test_unique_labels]
    plt.bar(labels_for_plot, test_counts)
    plt.xlabel("Class")
    plt.ylabel("Number of test images")
    plt.title("Test Set Label Distribution (Balanced)")
    plt.show()

    # Print counts and percentages for the balanced test set
    print("\nTest set counts and percentages:")
    for label, count in zip(test_unique_labels, test_counts):
         class_name = class_names[label] if 'class_names' in locals() else f"Label {label}"
         print(f"{class_name}: {count} images ({count/len(balanced_test_true_labels):.1%} of test set)")

else:
    print("Cannot perform a balanced split with less than two unique classes.")
# NB the test set must be splitted BEFORE oversampling to avoid data leakage!
# -------------------------------------------------------------------------
#from sklearn.model_selection import train_test_split
#returns numpy arrays containing the paths to images and the labels
train_images_paths, test_images_paths, train_true_labels, test_true_labels = train_test_split(
    images_paths_np,
    labels_np,
    test_size= 0.15,
    stratify=labels,
    random_state=42,
)

test_images_paths_np = np.array(test_images_paths)
test_true_labels_np = np.array(test_true_labels)
# print("train images paths:", train_images_paths)
# print("true test labels:", test_true_labels)
# # For the cross-validation, we'll use train_images_paths and labels_temp
train_images_paths_np = np.array(train_images_paths) #contains the images paths
train_labels_np = np.array(train_true_labels) #contains the labels
print(f"{train_images_paths_np.shape[0]} training images")
print(f"{len(test_images_paths)} test images")
#test_images_paths = [os.path.basename(path) for path in test_images_paths]
# print(test_images_paths)
print(type(train_images_paths))

unique_labels, counts = np.unique(test_true_labels_np, return_counts=True)


plt.bar([class_names[label] for label in unique_labels], counts)
plt.xlabel("Class")
plt.ylabel("Number of test images")
plt.title("Test Set Label Distribution")
plt.show()

for label, count in zip(unique_labels, counts):
    print(f"Label {label} ({class_names[label]}): {count} images")

for label, count in zip(unique_labels, counts):
    print(f"Label {label} ({class_names[label]}) is: {count/test_true_labels_np.shape[0]}")
def extract_patient_id(image_path):
    # Example: parse from the file name
    # In real code, you might have a different pattern
    match = re.search(r'(\d{4})', image_path)
    if match:
        return match.group(1)
    else:
        return "UNKNOWN"

# Build a DataFrame
df = pd.DataFrame({
    "image_path": images_paths_np,
    "label": labels_np
})

df["patient_id"] = df["image_path"].apply(extract_patient_id)

display(df)

# Ensure everything is string or int
df["patient_id"] = df["patient_id"].astype(str)

# Now group by patient to get a single label per patient.
# If every patient truly has exactly one label, we can just take .first()
patient_label_df = df.groupby("patient_id", as_index=False)["label"].first()

unique_pat_ids = patient_label_df["patient_id"].values  # need these to stratify for patient
print(f"Unique patient IDs: {unique_pat_ids}")
print(f"Number of unique patients: {len(unique_pat_ids)}")
pat_labels = patient_label_df["label"].values
print(f"Unique patient labels: {pat_labels}")

patient_label_df
yaml_path = f"/home/zano/Documents/TESI/TESI/configs/{num_input_channels}c/densenet121.yaml"
cfg = ConfigLoader(yaml_path) 
cfg.set_freezed_layer_index(None)
cfg.set_transfer_learning(False)
transfer_learning = cfg.get_transfer_learning()
pretrained_weights = None# "imagenet" if transfer_learning else None # 'microscopynet' or "imagenet"
model_library = "torchvision" # or "torchvision" or "monai"
color_transforms = False
train_transforms, val_transforms, test_transforms = tf.get_transforms(cfg,color_transforms=color_transforms)
model_manager = ModelManager(cfg, library=model_library)
# Verify the number of unique labels in the dataset
num_classes = len(np.unique(train_labels_np))
print(f"Number of classes in the dataset: {num_classes}")

# Ensure the model's output matches the number of classes
model, device = model_manager.setup_model(num_classes=num_classes, pretrained_weights=pretrained_weights)

print(model)
print(cfg.get_model_input_channels())

if model.__class__.__name__ == "ViT":
    print("Using ViT hence it requires custom training and validation functions")
    train_epoch = train_epoch_vit
    val_epoch = val_epoch_vit
    
        
def print_transform_names(compose):
    """
    Print the class names of the transformations inside a Compose object.

    Args:
        compose: A torchvision.transforms.Compose object.
    """
    for idx, transform in enumerate(compose.transforms):
        print(f"{idx + 1}. {transform.__class__.__name__}")

print_transform_names(train_transforms)
from utils.data_visualization_functions import visualize_tiff
# Example usage:
for i in range(len(balanced_test_images_paths)):
    print(f" Image{i} : {balanced_test_images_paths[i]}")
    visualize_tiff(balanced_test_images_paths[i])
from utils.data_visualization_functions import display_images_by_class
# from utils.filtering_functions import filter_paths_by_imageIds, filter_paths_by_classIndex

# filtered_paths, filtered_labels = filter_paths_by_classIndex(images_paths_np, labels_np, 0)
# indexes_to_exclude = [4121, 5358, 5435, 5745, 5717, 5767,5753,5878,5776,5978,6179,6053]
# filtered_paths, filtered_labels = filter_paths_by_imageIds(filtered_paths, filtered_labels, indexes_to_exclude)
# filtered_paths_x, filtered_labels_x = filtered_paths[1], filtered_labels[1]
# visualize_tiff(filtered_paths_x,channel_wise_norm=True)
# Display filtered images
class_names = [class0_name,class1_name]
display_images_by_class(images_paths_np, labels_np, class_names, ncols=5, normalize=True)
# test_msa_ids = [7144,7120,7239,7293]
# test_dir = os.path.join(data_dir, "TEST")
# test_imagesx_paths = list(sorted(glob.glob(os.path.join(test_dir, "*.tif"))))
# print(f"class 0:{class0_name} class 1:{class1_name}")

# #nice result if i flip the labels dont know why
# truex_test_labels = [get_label(path,test_msa_ids,test_class0_ids) for path in test_imagesx_paths]
# truex_test_labels = np.array(truex_test_labels)
# display_images_by_class(test_imagesx_paths, truex_test_labels, class_names, ncols=5, normalize=True, channel_wise_norm=True)
## simCLR
With a limited labeled dataset, achieving great generalization scores is difficult. SimCLR is a Self-Supervised Learning (SSL) approach to learn meaningful features directly from unlabeled data by contrasting augmented image pairs, without requiring explicit disease labels initially.

The goals are:

*   Learn robust, biologically relevant embeddings from your microscopy images.
*   Reduce overfitting by leveraging subtle structural patterns in data.

The implementation consists of:

*   A SimCLR dataset returning augmented pairs.
*   Using MONAI's contrastive loss to minimize the similarity between negative samples and maximize the similarity between positive samples (cosine similarity).
*   Training a DenseNet121-based SimCLR model, clearly defined with a modular encoder and projection head (the projection head has been shown to increase performance in the original paper).

Once the model is trained to minimize the contrastive loss (i.e., push embeddings of different subjects apart and push closer the ones of the same):

*   Replace the projection head with a linear classifier.
*   Fine-tune this network (pre-trained encoder + linear classifier) with the labeled data.

Hopefully, fine-tuning leverages embeddings previously learned by SSL, improving classification performance compared to standard supervised training or pure ImageNet transfer learning since the images on which it's trained are very different.

Before starting the fine-tuning, visualizing extracted features (e.g., via t-SNE) is useful to understand if the decoders learned meaningful features (if clusters can be detected).


### Problems of SimClr
* requires large batches to have good results --> high vram needed
*   Sensitive to the choice of data augmentations.
*   Contrastive loss can be computationally expensive.
*   May not work well if the unlabeled data is very different from the labeled data.
*   The quality of learned representations depends heavily on the quality and diversity of the unlabeled data.

yaml_path = f"/home/zano/Documents/TESI/TESI/configs/{num_input_channels}c/resnet18.yaml"
cfg = ConfigLoader(yaml_path) 
cfg.set_freezed_layer_index(None)
cfg.set_transfer_learning(False)
transfer_learning = cfg.get_transfer_learning()
pretrained_weights = None #"imagenet" if transfer_learning else None # 'microscopynet' or "imagenet"
model_library = "torchvision" # or "torchvision" or "monai"
color_transforms = False
train_transforms, val_transforms, test_transforms = tf.get_transforms(cfg,color_transforms=color_transforms)
model_manager = ModelManager(cfg, library=model_library)
# Verify the number of unique labels in the dataset
num_classes = len(np.unique(train_labels_np))
print(f"Number of classes in the dataset: {num_classes}")
import time
import torchvision.transforms.v2 as T
from lightly.data import LightlyDataset
from lightly.data.collate import SimCLRCollateFunction
import pytorch_lightning as pl
from lightly.loss import NTXentLoss
import torch.nn as nn
from pytorch_lightning import Trainer
from lightly.transforms import SimCLRTransform
from lightly.models.modules import SimCLRProjectionHead
from utils.train_functions import LinearProbeHead 
# from lightly.models.modules import SimCLRProjectionHead
torch.set_float32_matmul_precision('high')  # or 'high' for better performance to enable tensor cores?
pl.seed_everything(42)
ssl_input_dir = f"/home/zano/Documents/TESI/{num_input_channels}c_MIP_new/CONTROL"
# encoder = DenseNet121(
#     spatial_dims=2,
#     in_channels=3,
#     out_channels=128,  # final feature dim for dense layers
#     pretrained=False
# )

# # 1) define the encoder
# encoder, device = ModelManager(cfg, library=model_library).setup_model(
#     num_classes=128,  # not used in SSL
#     # pretrained_weights="imagenet",
# )
# Ensure the model's output matches the number of classes
encoder, device = model_manager.setup_model(num_classes=num_classes, pretrained_weights=pretrained_weights)
print(encoder)

print(f"encoder class: {encoder.__class__.__name__}")

class SimCLRModule(pl.LightningModule):
    def __init__(self, backbone, emb_dim=64, proj_hidden_dim=512, temperature=0.07, lr=3e-4):
        super().__init__()
        self.save_hyperparameters()
        self.backbone = backbone
        self.temperature = temperature
        self.lr = lr
        
        ##STRIP ENCODER LINEAR PROBE HEAD
        backbone_outdim = self._get_projection_head_input_dim()
        # print(self.backbone)
        self._remove_linear_probe_head()
        
        self.projection_head = SimCLRProjectionHead(
            input_dim=backbone_outdim,
            hidden_dim=proj_hidden_dim,
            output_dim=emb_dim,
        )

        self.criterion = NTXentLoss(temperature=self.temperature)
    
    @property
    def encoder(self):
        return self.backbone
        
    def _remove_linear_probe_head(self):
        # if isinstance(self.backbone.fc, LinearProbeHead):
        # print(self.backbone.fc.__class__.__name__)
        if hasattr(self.backbone, "fc"):
            if self.backbone.fc.__class__.__name__ == "LinearProbeHead":
                self.backbone.fc = nn.Identity()
            
        elif hasattr(self.backbone, "classifier"):
            if self.backbone.classifier.__class__.__name__ == "LinearProbeHead":
                self.backbone.classifier = nn.Identity()
        else:
            raise RuntimeError(
                    "Could not remove linear probe head"
                )
    
    def _get_projection_head_input_dim(self):
        """
        Return the dimensionality of the **feature vector that comes right
        before the classifier** (the value you must feed into a projection
        or linear-probe head).
        
        """
        print(f"backbone: {self.backbone}")
        print(f"hasattr(self.backbone, 'fc'): {hasattr(self.backbone, 'fc')}")
        print(f"hasattr(self.backbone, 'classifier'): {hasattr(self.backbone, 'classifier')}")
        print(f"hasattr(self.backbone, 'in_features'): {hasattr(self.backbone.fc, 'in_features')}")
        
        if hasattr(self.backbone, "fc") and hasattr(self.backbone.fc, "in_dim"):
            print(f"backbone.fc.in_features: {self.backbone.fc.in_dim}")
            if self.backbone.fc.__class__.__name__ == "LinearProbeHead":
                return self.backbone.fc.in_dim
            # return self.backbone.fc.in_features
        elif hasattr(self.backbone, "classifier"):
            if self.backbone.classifier.__class__.__name__ == "LinearProbeHead":
                return self.backbone.classifier.in_dim
            
        print(f" backbone input dim: {self.backbone.in_dim}")
        return self.backbone.in_dim
        
    def forward(self, x):
        feats = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(feats)
        return z

    def training_step(self, batch, batch_idx):
        (x0, x1),_,_ = batch #x0,x1 are tensor of shape (B,C,H,W)
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def configure_optimizers(self):
        # optim = torch.optim.SGD(self.parameters(), lr=self.lr) 
        optim = torch.optim.Adam(self.parameters(), lr=self.lr,weight_decay=1e-6)
        return optim
    
from lightly.transforms.simclr_transform import SimCLRTransform

STABLE_BACKBONE_PATH = f"simclr_encoder_weights_{num_input_channels}c_{encoder.__class__.__name__}.pth"
# 3) Create the SimCLR transform
transform = SimCLRTransform(
    input_size=256,
    gaussian_blur=0.5,  # probability of applying blur
    random_gray_scale = 0.2,  # probability of grayscale
    cj_strength=0.4, # color jitter strength
    vf_prob=0.5,
    hf_prob=0.5,
    rr_prob=0.5,
    rr_degrees=180,
)


dataset = LightlyDataset(
    input_dir=ssl_input_dir,
    transform=transform
)

# collate_fn = SimCLRCollateFunction(input_size=256)
print("len of dataset")
print(len(dataset)) 

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size = len(dataset),
    shuffle=True,
    # collate_fn=collate_fn,
    drop_last=True,
    num_workers=2
)

# 4) Create the module & trainer
model = SimCLRModule(
    backbone=encoder,
    emb_dim=128,
    proj_hidden_dim=512,
    temperature=0.07,
    lr=3e-4
)

trainer = pl.Trainer(
    max_epochs= 120,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    precision="16-mixed",
    enable_progress_bar=True,
    log_every_n_steps=1
)

start_time = time.time()
# 5) Fit
trainer.fit(model=model, train_dataloaders=loader)
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")
# 6) Save weights
STABLE_BACKBONE_PATH = f"simclr_encoder_weights_{num_input_channels}c_{encoder.__class__.__name__}.pth"

torch.save(
    model.backbone.state_dict(),
    STABLE_BACKBONE_PATH
)

print(f"Saved encoder weights to {STABLE_BACKBONE_PATH}")
import copy
# 7) Reload the backbone
encoder_type = "simclr"
simclr_encoder = copy.deepcopy(model.backbone) # Extract the encoder part not taking into consideration the projection head
# 3) Load your pre-trained SSL weights
state_dict = torch.load(STABLE_BACKBONE_PATH, map_location=device)
simclr_encoder.load_state_dict(state_dict)
simclr_encoder = simclr_encoder.to(device)
# 8) Use encoder for downstream tasks
# small test to see the encoder output dimension
from utils.train_functions import remove_projection_head
import copy
# 4) (Optional) Quick check of encoder output dim
encoder = copy.deepcopy(simclr_encoder).eval()  # Set the encoder to evaluation mode
print(cfg.get_image_shape())
with torch.no_grad():
    test_input = torch.zeros(1, 3, cfg.get_image_shape()[0], cfg.get_image_shape()[1], device=device)
    encoder_out = simclr_encoder(test_input).flatten(start_dim=1)
    model_without_projection = remove_projection_head(copy.deepcopy(simclr_encoder))  # Set the encoder to evaluation mode
    test_input = torch.zeros(1, 3, cfg.get_image_shape()[0], cfg.get_image_shape()[1], device=device)
    feats = model_without_projection(test_input).flatten(start_dim=1)
print(f"Detected encoder output dimension: {encoder_out.shape[1]}")
print(f"Detected encoder output dimension once removed classification head: {feats.shape[1]}")

print(simclr_encoder)
# encoder.train()  # Set the encoder back to training mode
# load the model weights
#LOAD WITH RANDOMLY INITIALIZED BACKBONE TO TEST IF SSL IS WORKING
simclr_encoder, device = model_manager.setup_model(num_classes=num_classes, pretrained_weights=pretrained_weights)
encoder_type = "NOSSL"

def _remove_linear_probe_head(backbone):
        # if isinstance(self.backbone.fc, LinearProbeHead):
        # print(self.backbone.fc.__class__.__name__)
        if hasattr(backbone, "fc"):
            if  backbone.fc.__class__.__name__ == "LinearProbeHead":
                print("removing linear probe head")
                backbone.fc = nn.Identity()
            
        elif hasattr(backbone, "classifier"):
            if backbone.classifier.__class__.__name__ == "LinearProbeHead":
                backbone.classifier = nn.Identity()
        else:
            raise RuntimeError(
                    "Could not remove linear probe head"
                )
_remove_linear_probe_head(simclr_encoder)
print(simclr_encoder)
### TRAINING LOOP
import copy
from utils.data_visualization_functions import generate_cv_results_figure
from utils.train_functions import  SSLClassifierModule

FREEZE_ENCODER = True  
bool_list = [True, False] 
for FREEZE_ENCODER in bool_list:
    def model_factory(lr: float)-> torch.nn.Module:
        """
        Model‐factory that instantiates a fresh SSLClassifierModule
        with the given learning rate.
        """
        fresh_encoder = copy.deepcopy(simclr_encoder)
        return SSLClassifierModule(
            encoder=fresh_encoder,     # capture this from your outer scope
            num_classes=2,
            freeze_encoder=FREEZE_ENCODER,
            lr=lr,
            backbone_output_dim=feats.shape[1],
            # input_shape=(1,3,256,256),  # Adjust as needed (B, C, H, W)
        )
        

    from classes.NestedCVStratifiedByPatient import NestedCVStratifiedByPatient
    # cfg.set_freezed_layer_index(None)
    experiment = NestedCVStratifiedByPatient(
        df=df, cfg=cfg, labels_np=labels_np, pat_labels=pat_labels, unique_pat_ids=unique_pat_ids,
        pretrained_weights = pretrained_weights,
        class_names = class_names, model_factory=model_factory, num_folds=6
    )
    # cfg.set_freezed_layer_index(None)
    # hold_out_cv = True
    using_cosine_scheduler = False

    per_fold_training_metrics, outer_fold_test_results = experiment.run_experiment()


    from utils.mlflow_functions import log_SSL_run_to_mlflow
    def get_best_fold_idx(outer_fold_test_results, metric="test_balanced_acc"):
        """
        Get the index of the best fold based on a specified metric.

        Args:
            outer_fold_test_results (list of dict): List containing test results for 
                each outer fold. Each element should be a dictionary with metrics 
                as keys.
            metric (str): The metric name to use for selecting the best fold. 
                Default is "test_balanced_acc".

        Returns:
            int: The index of the fold with the highest value for the specified metric.

        Example:
            best_idx = get_best_fold_idx(results, metric="test_f1")
        """

        print(outer_fold_test_results)
        best_fold_idx = np.argmax([r[metric] for r in outer_fold_test_results])
        # print(f"Best Balanced Accuracy Fold Index: {best_bac_fold_idx}")
        best_fold_result = outer_fold_test_results[best_fold_idx]
        print(f"Best {metric} Fold Result: {best_fold_result}")
        fold_idx = best_fold_result["fold"]
        return fold_idx

    #loading the best model for the metric selected, it's then used for computing gradcams during logging
    best_fold_idx = get_best_fold_idx(outer_fold_test_results, metric="test_balanced_acc")
    model_instance_for_logging, _ = experiment._get_model_and_device()
    model_instance_for_logging.eval()
    try:
        model_instance_for_logging.load_state_dict(torch.load(f"best_model_fold_{best_fold_idx}.pth"))
    except FileNotFoundError:
        raise FileNotFoundError("Could not find best_model_fold_{best_fold_idx}.pth")
    except Exception as e:
        raise Exception(f"Error loading model weights: {str(e)}")

    # Now, call the logging function:
    train_transforms, val_transforms,_ = experiment.get_current_fold_transforms()
    # Now, call the logging function:
      # or "moco" if using MoCo
    log_SSL_run_to_mlflow(
        environmentFlags=environment_flags,
        cfg=cfg,
        model=model_factory(0.001),  # Pass a dummy model to log the run
        class_names=class_names,
        fold_results=outer_fold_test_results,
        per_fold_metrics=per_fold_training_metrics,
        hold_out_cv=True,
        test_transforms=val_transforms,
        all_images_paths_np=images_paths_np,
        all_labels_np=labels_np,
        test_images_paths_np=test_images_paths_np,
        test_true_labels_np=test_true_labels_np,
        yaml_path=yaml_path,
        color_transforms=color_transforms,
        model_library=model_library,
        encoder_type= encoder_type,
        pretrained_backbone_path = STABLE_BACKBONE_PATH,
        train_transforms=train_transforms,
        # val_transforms=val_transforms,
        freeze_encoder=FREEZE_ENCODER,
        ssl=True,
    )
fold_6_val_image_paths = experiment.get_val_image_paths_per_fold(6)
print(fold_6_val_image_paths)

from utils.data_visualization_functions import visualize_tiff
# Example usage:
for i in range(len(fold_6_val_image_paths)):
    print(f" Image{i} : {fold_6_val_image_paths[i]}")
    visualize_tiff(fold_6_val_image_paths[i])